# covariates , tagsを作成する

In [1]:
# すでにcomplete_tags,complete_death_tagsがあるのでUNIQUE_IDとsample_idを合わせておわり

In [116]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import datetime
from datetime import timedelta
import numpy as np

from tqdm import tqdm
import time

file_path="E:/Data/Private/MIMIC_III/mimic_csv/"
outfile_path="E:/Data/Private/MIMIC_III/mimic_csv/Clean_data/"

In [117]:
df=pd.read_csv(outfile_path+'GRU_ODE_Dataset.csv')

In [118]:
tags=pd.read_csv(outfile_path+"complete_death_tags.csv")
covs=pd.read_csv(outfile_path+"complete_covariates.csv")

In [119]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [120]:
# ロード
d=pickle_load( outfile_path+'sample_unique_id_dict.pickle')

In [ ]:
# sample_id->unique_idをunique_id->sample_idに
# 
def get_swap_dict(d):
    return {v: k for k, v in d.items()}

d_swap = get_swap_dict(d)
print(d_swap)

# tags process

In [123]:
tags

,Unnamed: 0,HADM_ID,Value,UNIQUE_ID
0,21289,190677,0,0
1,22840,197233,0,1
2,23024,197967,0,2
3,5926,125265,0,3
4,4316,118320,0,4
...,...,...,...,...
21241,10664,145651,0,21241
21242,4769,120247,0,21242
21243,19271,182292,0,21243
21244,11399,148713,0,21244


In [124]:
if "Unnamed: 0" in tags.columns:
    tags=tags.drop("Unnamed: 0",axis=1)
if "HADM_ID" in tags.columns:
    tags=tags.drop("HADM_ID",axis=1)
tags

,Value,UNIQUE_ID
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
21241,0,21241
21242,0,21242
21243,0,21243
21244,0,21244


In [125]:
tags=tags.reindex(columns=['UNIQUE_ID', 'Value'])
tags

,UNIQUE_ID,Value
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
21241,21241,0
21242,21242,0
21243,21243,0
21244,21244,0


In [126]:
tags["UNIQUE_ID"]=tags["UNIQUE_ID"].map(d_swap)
tags=tags.rename(columns={'UNIQUE_ID' : 'ID' , "Value":"label"})

In [127]:
tags

,ID,label
0,7513,0
1,10115,0
2,259,0
3,20868,0
4,3090,0
...,...,...
21241,2343,0
21242,4206,0
21243,20925,0
21244,10515,0


In [128]:
tags.to_csv(outfile_path+"GRU_ODE_death_tags.csv",index=False)

In [129]:
tags.columns

Index(['ID', 'label'], dtype='object')

## Covs process

In [130]:
covs

,UNIQUE_ID,004,008,009,011,012,013,021,027,031,...,997,998,999,V45,V50,V55,V56,V58,V59,V71
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21241,21241,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21242,21242,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21243,21243,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21244,21244,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
covs["UNIQUE_ID"]=covs["UNIQUE_ID"].map(d_swap)
covs=covs.rename(columns={'UNIQUE_ID' : 'ID'})

In [132]:
covs

,ID,004,008,009,011,012,013,021,027,031,...,997,998,999,V45,V50,V55,V56,V58,V59,V71
0,7513,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20868,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3090,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21241,2343,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21242,4206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21243,20925,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21244,10515,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
covs.to_csv(outfile_path+"GRU_ODE_covariates.csv",index=False)

In [135]:
import torch
print(torch.cuda.is_available())

True


In [136]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(dev)


cuda:0


In [137]:
torch.cuda.get_device_name(0)

'GeForce RTX 2080 Ti'

In [138]:
 torch.backends.cudnn.version()

7604

In [151]:
m = torch.nn.Sigmoid()
input = torch.randn(3,1)
output = m(input)

In [152]:
input

tensor([[-0.8295],
        [-0.3264],
        [-2.2645]])

In [153]:
output

tensor([[0.3038],
        [0.4191],
        [0.0941]])

In [154]:
input = torch.randn(3,1)
output = torch.sigmoid(input)

In [155]:
output

tensor([[0.5179],
        [0.4571],
        [0.6273]])

In [156]:
output.shape

torch.Size([3, 1])